<a href="https://colab.research.google.com/github/Abhiyantrana-Navonmesakah/Saanjh-sahayak/blob/main/biomed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install open_clip_torch==2.23.0 transformers==4.35.2 matplotlib

In [ ]:
pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 3.1 MB/s eta 0:00:00


In [ ]:
9from urllib.request import urlopen
from PIL import Image
import torch
from huggingface_hub import hf_hub_download
from open_clip import create_model_and_transforms, get_tokenizer
from open_clip.factory import HF_HUB_PREFIX, _MODEL_CONFIGS

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [ ]:
pip install huggingface_hub

In [ ]:
from groq import Groq
def analysize(text):
  client = Groq(
      api_key="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",
  )

  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": "you are a medical health analyzer based on the health labels of zero-shot classification model which givers value between 0 to 1 for each label . you will be provided all the labels and their values to you . you are a extreme medical knowlege expert analyze all the health labels and provide a suggestive and generalized analysis. also mention if anything is serious in the report and it must be in one or two paragraphs precise. health labels are "+text,
          }
      ],
      model="llama3-8b-8192",
  )

  return (chat_completion.choices[0].message.content)

In [ ]:
analysize("hello hi")

'I cannot provide a medical diagnosis or analysis based on the labels "hello" and "hi".'

In [ ]:
from huggingface_hub import hf_hub_download

In [ ]:
# Download the model and config files
hf_hub_download(
    repo_id="microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224",
    filename="open_clip_pytorch_model.bin",
    local_dir="checkpoints"
)
hf_hub_download(
    repo_id="microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224",
    filename="open_clip_config.json",
    local_dir="checkpoints"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


open_clip_pytorch_model.bin:   0%|          | 0.00/784M [00:00<?, ?B/s]

open_clip_config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

'checkpoints/open_clip_config.json'

In [ ]:
import json
model_name = "biomedclip_local"

with open("checkpoints/open_clip_config.json", "r") as f:
    config = json.load(f)
    model_cfg = config["model_cfg"]
    preprocess_cfg = config["preprocess_cfg"]

In [ ]:
if (not model_name.startswith(HF_HUB_PREFIX)
    and model_name not in _MODEL_CONFIGS
    and config is not None):
    _MODEL_CONFIGS[model_name] = model_cfg

tokenizer = get_tokenizer(model_name)
model, _, preprocess = create_model_and_transforms(
    model_name=model_name,
    pretrained="checkpoints/open_clip_pytorch_model.bin",
    **{f"image_{k}": v for k, v in preprocess_cfg.items()},
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/225k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/open_clip/factory.py:129: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you s

In [ ]:

# Zero-shot image classification
template = 'this is a photo of '
labels = [
    'adenocarcinoma histopathology',
    'brain MRI',
    'covid line chart',
    'squamous cell carcinoma histopathology',
    'immunohistochemistry histopathology',
    'bone X-ray',
    'chest X-ray',
    'pie chart',
    'hematoxylin and eosin histopathology'
]

dataset_url = 'https://huggingface.co/microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224/resolve/main/example_data/biomed_image_classification_example_data/'
test_imgs = [
    'squamous_cell_carcinoma_histopathology.jpeg',
    'H_and_E_histopathology.jpg',
    'bone_X-ray.jpg',
    'adenocarcinoma_histopathology.jpg',
    'covid_line_chart.png',
    'IHC_histopathology.jpg',
    'chest_X-ray.jpg',
    'brain_MRI.jpg',
    'pie_chart.png'
]


In [ ]:
def load_local_images(image_paths):
    images = []
    for path in image_paths:
        img = Image.open(path)
        processed_img = preprocess(img)  # Using the model's preprocess function
        images.append(processed_img)
    return torch.stack(images).to(device)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.eval()

def process_data(images, labels, template):
    # Prepare text prompts
    text_prompts = [template + l for l in labels]

    # Tokenize text
    context_length=77
    texts = tokenizer(text_prompts, context_length=context_length).to(device)

    # Get predictions
    with torch.no_grad():
        image_features, text_features, logit_scale = model(images, texts)
        logits = (logit_scale * image_features @ text_features.t()).detach().softmax(dim=-1)

    return logits

In [ ]:
# Example of using your own images and labels
def classify_custom_images(image_paths, custom_labels):
    # Load and preprocess images
    custom_images = load_local_images(image_paths)

    # Process data
    logits = process_data(custom_images, custom_labels, template)

    # Get predictions
    predictions = torch.argmax(logits, dim=-1)

    # Print results
    for i, path in enumerate(image_paths):
        print(f"Image: {path}")
        print(f"Predicted: {custom_labels[predictions[i]]}")
        top_3 = torch.topk(logits[i], len(custom_labels))
        ans=[]
        for score, idx in zip(top_3.values, top_3.indices):
            ans.append(f"{custom_labels[idx]}: {score:.3f}")
        return ans

In [ ]:
my_images = [
    '/content/1.jpg',

        ]

my_labels = [
          'aneurysm',
          'glioma',
          'notumour',
          'pituitary',
    ]
classify_custom_images(my_images, my_labels)

Image: /content/1.jpg
Predicted: pituitary
pituitary: 0.529
glioma: 0.471
aneurysm: 0.000
notumour: 0.000


In [ ]:
!pip install pyngrok
!pip install flask

from flask import Flask
from pyngrok import ngrok

In [ ]:

pip install open_clip_torch==2.23.0 transformers==4.35.2 matplotlib


In [ ]:
from flask import Flask, request, jsonify
from PIL import Image
from io import BytesIO
import base64

In [ ]:
ngrok.set_auth_token("2XvoAEnwqtCypdQj1fiXsRnzrnu_3odZpjm4kZyTGm6c7cCih")

In [ ]:
import os

In [ ]:
app = Flask(__name__)

In [ ]:
@app.route("/")
def home():
    return "BiomedCLIP Classification Server"

@app.route('/getImage', methods=['POST'])
def getImage():
    try:
        data = request.get_json()

        if not data or 'photo' not in data:
            return jsonify({'error': 'No photo data received'}), 400

        # Extract base64 data
        photo_data = data['photo']
        imageType=data['imageType']
        print(imageType)

        my_images = [
    '/content/Image.png'
        ]

        my_labels = []
        if(imageType=='brain'):
          my_labels=[
              'Normal Brain',
              'Brain Tumor',
              'Stroke',
              'Brain Swelling',
              'Glioblastoma Multiforme (GBM)',
              'Meningioma',
              'Cerebral Microhemorrhages',
              'Posterior Reversible Encephalopathy Syndrome (PRES)',
              'Multiple Sclerosis Lesions'
          ]
        elif(imageType=='bones'):
          my_labels=[
              'Normal Bone',
              'Fracture',
              'Arthritis',
              'Joint Dislocation',
              'Osteosarcoma',
              'Pagets Disease',
              'Avascular Necrosis',
              'Metastatic Bone Disease',
              'Fibrous Dysplasia'
          ]
        elif(imageType=='lungs'):
          my_labels=[
              'Clear lung fields',
              'Bronchiectasis'
              'Visible normal blood vessels',
              'Fluid in lungs',
              'Loss of lung volume',
              'Honeycombing',
              'Architectural distortion',
              'Solitary pulmonary nodules'
          ]
        elif(imageType=='heart'):
          my_labels=[
              'Normal Heart',
              'Enlarged Heart',
              'Heart Attack Evidence',
              'Valve Problems',
              'Left Ventricular Hypertrophy (LVH)',
              'Hypertrophic Cardiomyopathy',
              'Cardiac Amyloidosis',
              'Constrictive Pericarditis',
              'Takotsubo Cardiomyopathy'
          ]
        else:
          my_labels=[
              'healthy report',
              'unhealthy report'
          ]

# Classify your images


        base64_string = photo_data
        image_data = base64.b64decode(base64_string)
        image = Image.open(BytesIO(image_data))
        image.save("Image.png")

        ans=classify_custom_images(my_images, my_labels)
        analysis=analysize(",".join(ans))

        return jsonify({
                'metrics': ans,
                'analysis':analysis,

            })


        return jsonify({
            'error': 'Invalid image data format'
        }), 400

    except Exception as e:
        return jsonify({'error': str(e)}), 500

public_url = ngrok.connect(5000)
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000/\"")

if __name__ == "__main__":
    app.run(debug=False, port=5000)  # Set debug=False for production

 * ngrok tunnel "NgrokTunnel: "https://71d4-35-231-49-232.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000/"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


brain
Image: /content/Image.png
Predicted: Glioblastoma Multiforme (GBM)


INFO:werkzeug:127.0.0.1 - - [22/Nov/2024 06:15:05] "POST /getImage HTTP/1.1" 200 -


brain
Image: /content/Image.png
Predicted: Glioblastoma Multiforme (GBM)


INFO:werkzeug:127.0.0.1 - - [22/Nov/2024 07:00:47] "POST /getImage HTTP/1.1" 200 -
